In [1]:
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 1.7 MB/s eta 0:00:00


In [2]:
import json
import pickle
import pandas as pd
import numpy as np
from getpass import getpass
from tqdm import tqdm
from typing import List

from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [3]:
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
llama3_70b_temp0 = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    api_key=GROQ_API_KEY)

# Test
llama3_70b_temp0.invoke("What is the capital of France?")

AIMessage(content='The capital of France is Paris.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 17, 'total_tokens': 25, 'completion_time': 0.022857143, 'prompt_time': 0.004994294, 'queue_time': None, 'total_time': 0.027851437}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_7ab5f7e105', 'finish_reason': 'stop', 'logprobs': None}, id='run-d1347781-0617-4f11-bdd6-29ab01ff0d89-0')

In [6]:
df = pd.read_csv("/content/dataset_concurso_rag.csv")
df.columns, len(df)

(Index(['pergunta', 'resposta', 'comentarios', 'numero_na_captura', 'id',
        'area', 'ano', 'banca', 'prova', 'duvidou_do_gabarito', 'enunciado',
        'assertivas', 'alternativas', 'classificacao',
        'artigos_nos_comentarios', 'artigos_completos', 'artigo_inventado',
        'topicos_para_busca', 'artigos_candidatos', 'frase_assunto_da_questao',
        'rag_art_inventado_full', 'rag_art_inventado_full_sem_art',
        'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos',
        'rag_artigos_candidatos_grupos'],
       dtype='object'),
 37)

In [7]:
df[['rag_art_inventado_full', 'rag_art_inventado_full_sem_art', 'rag_assunto_full', 'rag_topicos_full', 'rag_artigos_candidatos', 'rag_artigos_candidatos_grupos']]

,rag_art_inventado_full,rag_art_inventado_full_sem_art,rag_assunto_full,rag_topicos_full,rag_artigos_candidatos,rag_artigos_candidatos_grupos
0,"Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.238. Aquele que, por quinze anos, sem i...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.210. O possuidor tem direito a ser mant...,Art. 1.815. A exclusão do herdeiro ou legatári...,Art. 1.814. São excluídos da sucessão os herde...
1,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.784. Aberta a sucessão, a herança trans...",Art. 1.862. São testamentos ordinários:I - o p...,Art. 1.862. São testamentos ordinários:\nI - o...
2,Art. 1.876. O testamento particular pode ser e...,Art. 1.876. O testamento particular pode ser e...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 212. Salvo o negócio a que se impõe forma...,"Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 1.862. São testamentos ordinários:\nI - o...
3,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 1.976. O testador pode nomear um ou mais ...,Art. 92. Principal é o bem que existe sobre si...,"Art. 1.848. Salvo se houver justa causa, decla...","Art. 1.860. Além dos incapazes, não podem test..."
4,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,Art. 1.723. É reconhecida como entidade famili...,"Art. 1.784. Aberta a sucessão, a herança trans..."
5,"Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...","Art. 1.022. A sociedade adquire direitos, assu...",Art. 986. Enquanto não inscritos os atos const...,"Art. 1.793. O direito à sucessão aberta, bem c...",Art. 1.791. A herança defere-se como um todo u...
6,"Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.596. Os filhos, havidos ou não da relaç...","Art. 1.658. No regime de comunhão parcial, com...","Art. 1.639. É lícito aos nubentes, antes de ce...",Art. 1.576. A separação judicial põe termo aos...,"Art. 1.596. Os filhos, havidos ou não da relaç..."
7,Art. 1.997. A herança responde pelo pagamento ...,Art. 1.997. A herança responde pelo pagamento ...,"Art. 789. Nos seguros de pessoas, o capital se...","Art. 818. Pelo contrato de fiança, uma pessoa ...",Art. 1.792. O herdeiro não responde por encarg...,Art. 1.791. A herança defere-se como um todo u...
8,Art. 26. Decorrido um ano da arrecadação dos b...,Art. 26. Decorrido um ano da arrecadação dos b...,"Art. 1.824. O herdeiro pode, em ação de petiçã...","Art. 1.824. O herdeiro pode, em ação de petiçã...",Art. 1.813. Quando o herdeiro prejudicar os se...,Art. 1.814. São excluídos da sucessão os herde...
9,"Art. 1.857. Toda pessoa capaz pode dispor, por...","Art. 1.857. Toda pessoa capaz pode dispor, por...",Art. 26. Decorrido um ano da arrecadação dos b...,Art. 1.798. Legitimam-se a suceder as pessoas ...,Art. 1.830. Somente é reconhecido direito suce...,Art. 1.845. São herdeiros necessários os desce...


In [8]:
def loop_fn(df, fn, kwarg=None):
    list_of_outputs = []
    for index, row in tqdm(df.iterrows(), total=len(df)):
        try:
            if kwarg:
                output = fn(row, kwarg)
            else:
                output = fn(row)
            print(output)
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            output = "Error"
        list_of_outputs.append(output)
    return list_of_outputs

In [9]:
class GET_CONTEXT_JSON(BaseModel):
    analise: str = Field(description="Análise da questão, resposta e artigos de contexto.")
    parte_dos_artigos: List[str] = Field(description="Porçao ou trecho do artigo que ajuda a responder a pergunta.")
    artigos_id: List[str] = Field(description="Nome do artigos cujos trechos respondem a pergunta.")
    justificativa_de_cada_trecho: List[str] = Field(description="Razões pelas quais os trechos ajudam a responder a pergunta.")
    informacao_faltante: str = Field(description="Informação faltante que impede a resolução correta da questão, se houver.")
    resposta_obtida: str = Field(description="Resposta obtida pela análise dos trechos selecionados.")
    alternativa_resposta_obtida: str = Field(description="Alternativa da resposta obtida: A, B, C, D ou E.")
    resposta_correta: str = Field(description="Sim, se a resposta obtida for igual a resposta oficial.")


output_parser = JsonOutputParser(pydantic_object=GET_CONTEXT_JSON)

In [10]:
prompt = """
Você é professor de Direito e ávido conhecedor do Código Civil Brasileiro.
Use seus conhecimentos em leis e códigos analisar os itens da questão.

Em seguida selecione trechos dos artigos que melhor resolvem as sentenças da questão.

Irei fornecer uma lista de possíveis artigos do Código Civil e uma outra lista de artigos
citados por estudantes.

Os trechos devem ser usados para CONFIRMAR ou NEGAR as sentenças da questão.

Depois de gerar os trechos, análise se eles ajudam a chegar na resposta correta.
Caso não possibilitem chegar na mesma resposta, indique que informação está faltando.

questão: {questão}
resposta correta: {resposta}
artigos estudantes: {artigos_estudantes}
artigos possíveis: {artigos_da_busca}

Gere somente as informações dos campos do JSON, sem introduções ou formalismos.
Instruções de saída: {format_instructions}
""".strip()

prompt_template = PromptTemplate(
    input_variables=["questão", "resposta", "artigos_estudantes", "artigos_da_busca"],
    template=prompt,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain = prompt_template | llama3_70b_temp0 | output_parser

In [12]:
def call_context_generator(sample, rag_column):
    output = chain.invoke(
        {"questão": sample['pergunta'],
         "resposta": sample['resposta'],
         "artigos_estudantes": sample['artigos_completos'],
         "artigos_da_busca": sample[rag_column],
         })
    return output

# test
out = call_context_generator(df.iloc[15], rag_column='rag_artigos_candidatos')
out

{'analise': 'Análise da questão de direito de família sobre alimentos.',
 'parte_dos_artigos': ['O direito à prestação de alimentos é recíproco entre pais e filhos, e extensivo a todos os ascendentes, recaindo a obrigação nos mais próximos em grau, uns em falta de outros.',
  'Se o parente, que deve alimentos em primeiro lugar, não estiver em condições de suportar totalmente o encargo, serão chamados a concorrer os de grau imediato;'],
 'artigos_id': ['Art. 1.696', 'Art. 1.698'],
 'justificativa_de_cada_trecho': ['Este trecho confirma que os alimentos podem ser cobrados dos avós da criança, caso o genitor ou a genitora não estejam em condições de pagar alimentos.',
  'Este trecho reforça que, se o parente que deve alimentos em primeiro lugar não estiver em condições de suportar totalmente o encargo, os de grau imediato serão chamados a concorrer.'],
 'informacao_faltante': '',
 'resposta_obtida': 'Sim, os alimentos podem ser cobrados dos avós da criança, caso o genitor ou a genitora nã

In [13]:
list_of_outputs = loop_fn(df, call_context_generator, kwarg='rag_artigos_candidatos')

  3%|▎         | 1/37 [00:02<01:20,  2.23s/it]

{'analise': 'A questão envolve a análise dos direitos de Diego, herdeiro aparente e de boa-fé, em relação à herança deixada por seu pai Euclides.', 'parte_dos_artigos': ['Art. 1.214. O possuidor de boa-fé tem direito, enquanto ela durar, aos frutos percebidos.', 'Art. 1.219. O possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, bem como, quanto às voluptuárias, se não lhe forem pagas, a levantá-las, quando o puder sem detrimento da coisa, e poderá exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'], 'artigos_id': ['Art. 1.214', 'Art. 1.219'], 'justificativa_de_cada_trecho': ['O art. 1.214 estabelece que o possuidor de boa-fé tem direito aos frutos percebidos enquanto durar a boa-fé.', 'O art. 1.219 estabelece que o possuidor de boa-fé tem direito à indenização das benfeitorias necessárias e úteis, e pode exercer o direito de retenção pelo valor das benfeitorias necessárias e úteis.'], 'informacao_faltante': '', 'resposta_obti

  5%|▌         | 2/37 [00:04<01:27,  2.50s/it]

{'analise': 'Análise da questão sobre a sucessão testamentária.', 'parte_dos_artigos': ['Art. 1.867. Ao cego só se permite o testamento público, que lhe será lido, em voz alta, duas vezes, uma pelo tabelião ou por seu substituto legal, e a outra por uma das testemunhas, designada pelo testador, fazendo-se de tudo circunstanciada menção no testamento.', 'Art. 1.880. O testamento particular pode ser escrito em língua estrangeira, contanto que as testemunhas a compreendam.'], 'artigos_id': ['Art. 1.867', 'Art. 1.880'], 'justificativa_de_cada_trecho': ['Este artigo confirma que o testamento público é permitido para os cegos, o que justifica a afirmação I.', 'Este artigo permite que o testamento particular seja escrito em língua estrangeira, desde que as testemunhas a compreendam, o que justifica a afirmação IV.'], 'informacao_faltante': '', 'resposta_obtida': 'As afirmativas I e IV estão corretas.', 'alternativa_resposta_obtida': 'B', 'resposta_correta': 'Sim'}


  8%|▊         | 3/37 [00:30<07:16, 12.83s/it]

...

{'analise': 'A questão afirma que a legítima dos herdeiros necessários não pode ser incluída no testamento.', 'parte_dos_artigos': ['A legítima dos herdeiros necessários não poderá ser incluída no testamento.'], 'artigos_id': ['Art. 1.857'], 'justificativa_de_cada_trecho': ['Este trecho do artigo 1.857 explicitamente afirma que a legítima dos herdeiros necessários não pode ser incluída no testamento.'], 'informacao_faltante': '', 'resposta_obtida': 'Certo', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


 97%|█████████▋| 36/37 [11:33<00:20, 20.65s/it]

{'analise': 'A questão trata de uma ação de investigação de paternidade post mortem cumulada com petição de herança, e a pretensão de Renato referente à petição de herança.', 'parte_dos_artigos': ['A prescrição ocorre em dez anos, quando a lei não lhe haja fixado prazo menor.', 'decorridos cinco anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, se localizados nas respectivas circunscrições, incorporando-se ao domínio da União quando situados em território federal.'], 'artigos_id': ['Art. 205', 'Art. 1.822'], 'justificativa_de_cada_trecho': ['O art. 205 estabelece o prazo prescricional de 10 anos para a petição de herança, contado da abertura da sucessão.', 'O art. 1.822 estabelece que, decorridos 5 anos da abertura da sucessão, os bens arrecadados passarão ao domínio do Município ou do Distrito Federal, ou à União, se situados em território federal.'], 'informacao_faltante': '', 'resposta_obtida': 'A pretensão de Renato referente

100%|██████████| 37/37 [11:50<00:00, 19.20s/it]

{'analise': 'A questão apresenta uma situação hipotética em que João morre e sua esposa Ana está grávida. A pergunta é se o nascituro tem legitimidade para suceder.', 'parte_dos_artigos': ['Legitimam-se a suceder as pessoas nascidas ou já concebidas no momento da abertura da sucessão.'], 'artigos_id': ['Art. 1.798.'], 'justificativa_de_cada_trecho': ['Este artigo estabelece que as pessoas concebidas no momento da abertura da sucessão têm legitimidade para suceder.'], 'informacao_faltante': '', 'resposta_obtida': 'Certo', 'alternativa_resposta_obtida': 'C', 'resposta_correta': 'Sim'}


In [16]:
df_rag_artigos_candidatos = pd.DataFrame(list_of_outputs)
df_rag_artigos_candidatos.to_csv("df_rag_artigos_candidatos.csv", index=False)

In [17]:
df_rag_artigos_candidatos

,analise,parte_dos_artigos,artigos_id,justificativa_de_cada_trecho,informacao_faltante,resposta_obtida,alternativa_resposta_obtida,resposta_correta
0,A questão envolve a análise dos direitos de Di...,[Art. 1.214. O possuidor de boa-fé tem direito...,"[Art. 1.214, Art. 1.219]",[O art. 1.214 estabelece que o possuidor de bo...,,Diego tem direito a todos os frutos percebidos...,C,Sim
1,Análise da questão sobre a sucessão testamentá...,[Art. 1.867. Ao cego só se permite o testament...,"[Art. 1.867, Art. 1.880]",[Este artigo confirma que o testamento público...,,As afirmativas I e IV estão corretas.,B,Sim
2,A questão apresenta um caso de testamento feit...,[Art. 1.880. O testamento particular pode ser ...,"[Art. 1.880, Art. 1.858, Art. 1.876]",[O artigo 1.880 confirma que o testamento pode...,,O testamento pode ser escrito em língua estran...,E,Sim
3,A questão apresenta uma situação hipotética em...,"[Art. 1.860. Além dos incapazes, não podem tes...","[Art. 1.860, Art. 1.799, Art. 1.857]",[O art. 1.860 estabelece que os relativamente ...,,A incapacidade relativa de Mariana no momento ...,C,Sim
4,A questão apresenta uma situação hipotética em...,[§ 1 o A união estável não se constituirá se o...,[Art. 1.723],[Este trecho do art. 1.723 estabelece que a un...,,"Sim, é possível a configuração da união estáve...",C,Sim
5,A questão discute a responsabilidade dos herde...,[nenhum destes será obrigado a pagar senão a q...,[Art. 276],[Este trecho do Art. 276 indica que os herdeir...,,E,E,Sim
6,A questão apresenta um caso de divórcio culpos...,[Art. 1.609. O reconhecimento dos filhos havid...,"[Art. 1.609, Art. 1.601]",[O art. 1.609 estabelece que o reconhecimento ...,,Para a ação negatória de paternidade cumulada ...,E,Sim
7,A questão discute a responsabilidade solidária...,[salvo se a obrigação for indivisível],[Art. 270],"[Este trecho do art. 270 indica que, se um dos...",,Errado,E,Sim
8,"A questão apresenta uma situação em que Diana,...",[Quando o herdeiro prejudicar os seus credores...,[Art. 1.813],[Este trecho permite que os credores aceitem a...,,O Banco XYZ pode aceitar a herança e receber u...,D,Sim
9,Análise da questão e resposta com base nos art...,"[O testamento é ato personalíssimo, podendo se...","[Art. 1.858, Art. 1.860, Art. 1.786, Art. 1.845]",[Confirma que o testamento é ato personalíssim...,,"B) O testamento é ato personalíssimo, podendo ...",B,Sim
